In [1]:
import pandas as pd
import os
from datetime import datetime
import random
from glob import glob
import time
import numpy as np
from distutils.dir_util import copy_tree
import math

In [2]:
location_path = "regions/egypt/locations/route_0"
currenct_trainer_path = "trainers/atlas"


In [ ]:

class game:
    def __init__(self, trainer_path, location_path):
        self.load_data(trainer_path, location_path)
        self.load_values()
        
        
    def load_values(self):
        self.MAX_EV = 1024
        self.MAX_IV = 32
        self.MAX_AV = 256
        self.MAX_LR = 3
        self.TASK_DUR = 300
        
        
    def load_data(self, trainer_path, location_path):
        self.trainer_path = trainer_path
        self.location_path = location_path
        self.location_name = location_path.replace("regions/", "").replace("locations", "").replace("/", " ")
        self.trainer_name = trainer_path.replace("trainers/", "").replace("/", " ")
        if not(os.path.isdir(trainer_path)):
            print("creating new trainer!")
            time.sleep(4)
            self.create_trainer()
        self.trainer_pokemons = pd.read_csv(trainer_path + "/pokemons.csv", index_col=[0])
        self.trainer_items = pd.read_csv(trainer_path + "/items.csv", index_col=[0])
        self.wilderness_pokemons = pd.read_csv(location_path + "/wilderness_pokemons.csv", index_col=[0])
        self.wilderness_items = pd.read_csv(location_path + "/wilderness_items.csv", index_col=[0])
        self.tasks_csv = pd.read_csv(trainer_path + "/tasks.csv", index_col=[0])
        self.pokedex = pd.read_csv("pokedex.csv", index_col=[0])
        self.item_dex = pd.read_csv("item_dex.csv", index_col=[0])
        self.save_state()
        
        
    def create_trainer(self):
        copy_tree("trainers/proxy", self.trainer_path)
        
    def save_state(self):
        self.pokedex.sort_values(by=["species"], inplace=True)
        self.pokedex.reset_index(drop=True, inplace=True)
        self.pokedex["base_total"] = self.pokedex.base_hp + self.pokedex.base_attack + self.pokedex.base_defense + self.pokedex.base_ability + self.pokedex.base_resistance + self.pokedex.base_speed
        self.pokedex["total_base_utils"] = self.pokedex.base_charm + self.pokedex.base_search + self.pokedex.base_fly + self.pokedex.base_dig + self.pokedex.base_picking + self.pokedex.base_swim + self.pokedex.base_shaking
        self.item_dex.sort_values(by=["item_category"], inplace=True)
        self.item_dex.reset_index(drop=True, inplace=True)
        self.trainer_pokemons.to_csv(self.trainer_path + "/pokemons.csv", index=[0])
        self.trainer_items.to_csv(self.trainer_path + "/items.csv", index=[0])
        self.wilderness_pokemons.to_csv(self.location_path + "/wilderness_pokemons.csv", index=[0])
        self.wilderness_items.to_csv(self.location_path + "/wilderness_items.csv", index=[0])
        self.tasks_csv.to_csv(self.trainer_path + "/tasks.csv", index=[0])
        self.pokedex.to_csv("pokedex.csv", index=[0])
        self.item_dex.to_csv("item_dex.csv", index=[0])
        
    
    def refresh_trainer_pokemons(self):
        print("recalculating stats and levels!")
        time.sleep(1)

        self.trainer_pokemons["base_total"] = (self.pokedex.base_hp + self.pokedex.base_attack + self.pokedex.base_defense + self.pokedex.base_ability +
                                      self.pokedex.base_resistance + self.pokedex.base_speed)
        self.trainer_pokemons["total_base_utils"] = (self.pokedex.base_charm + self.pokedex.base_search + self.pokedex.base_fly + self.pokedex.base_dig +
                                            self.pokedex.base_picking + self.pokedex.base_swim + self.pokedex.base_shaking)
        self.trainer_pokemons.LR = self.trainer_pokemons.LR.apply(lambda x: 1 if pd.isnull(x) else x)
        
        self.trainer_pokemons.IV_hp = self.trainer_pokemons.IV_hp.apply(lambda x: random.randint(0,self.MAX_IV) if pd.isnull(x) else x)
        self.trainer_pokemons.IV_attack = self.trainer_pokemons.IV_attack.apply(lambda x: random.randint(0,self.MAX_IV) if pd.isnull(x) else x)
        self.trainer_pokemons.IV_defense = self.trainer_pokemons.IV_defense.apply(lambda x: random.randint(0,self.MAX_IV) if pd.isnull(x) else x)
        self.trainer_pokemons.IV_ability = self.trainer_pokemons.IV_ability.apply(lambda x: random.randint(0,self.MAX_IV) if pd.isnull(x) else x)
        self.trainer_pokemons.IV_resistance = self.trainer_pokemons.IV_resistance.apply(lambda x: random.randint(0,self.MAX_IV) if pd.isnull(x) else x)
        self.trainer_pokemons.IV_speed = self.trainer_pokemons.IV_speed.apply(lambda x: random.randint(0,self.MAX_IV) if pd.isnull(x) else x)
        self.trainer_pokemons.total_IV = self.trainer_pokemons["total_IV"] = (self.trainer_pokemons.IV_hp + self.trainer_pokemons.IV_attack
                                                                                + self.trainer_pokemons.IV_defense + self.trainer_pokemons.IV_ability +
                                                                                self.trainer_pokemons.IV_resistance + self.trainer_pokemons.IV_speed)
        
        self.trainer_pokemons.EV_hp = self.trainer_pokemons.EV_hp.apply(lambda x: 0 if pd.isnull(x) else x)
        self.trainer_pokemons.EV_attack = self.trainer_pokemons.EV_attack.apply(lambda x: 0 if pd.isnull(x) else x)
        self.trainer_pokemons.EV_defense = self.trainer_pokemons.EV_defense.apply(lambda x: 0 if pd.isnull(x) else x)
        self.trainer_pokemons.EV_ability = self.trainer_pokemons.EV_ability.apply(lambda x: 0 if pd.isnull(x) else x)
        self.trainer_pokemons.EV_resistance = self.trainer_pokemons.EV_resistance.apply(lambda x: 0 if pd.isnull(x) else x)
        self.trainer_pokemons.EV_speed = self.trainer_pokemons.EV_speed.apply(lambda x: 0 if pd.isnull(x) else x)
        self.trainer_pokemons.total_EV = self.trainer_pokemons["total_EV"] = (self.trainer_pokemons.EV_hp + self.trainer_pokemons.EV_attack
                                                                                + self.trainer_pokemons.EV_defense + self.trainer_pokemons.EV_ability +
                                                                                self.trainer_pokemons.EV_resistance + self.trainer_pokemons.EV_speed)
        
        self.trainer_pokemons.AV_hp = self.trainer_pokemons.AV_hp.apply(lambda x: 0 if pd.isnull(x) else x)
        self.trainer_pokemons.AV_attack = self.trainer_pokemons.AV_attack.apply(lambda x: 0 if pd.isnull(x) else x)
        self.trainer_pokemons.AV_defense = self.trainer_pokemons.AV_defense.apply(lambda x: 0 if pd.isnull(x) else x)
        self.trainer_pokemons.AV_ability = self.trainer_pokemons.AV_ability.apply(lambda x: 0 if pd.isnull(x) else x)
        self.trainer_pokemons.AV_resistance = self.trainer_pokemons.AV_resistance.apply(lambda x: 0 if pd.isnull(x) else x)
        self.trainer_pokemons.AV_speed = self.trainer_pokemons.AV_speed.apply(lambda x: 0 if pd.isnull(x) else x)
        self.trainer_pokemons.total_AV = self.trainer_pokemons["total_AV"] = (self.trainer_pokemons.AV_hp + self.trainer_pokemons.AV_attack
                                                                                + self.trainer_pokemons.AV_defense + self.trainer_pokemons.AV_ability +
                                                                                self.trainer_pokemons.AV_resistance + self.trainer_pokemons.AV_speed)
        
        
        self.trainer_pokemons.final_hp = np.floor((self.trainer_pokemons.level/100.0) * (np.floor(2*self.trainer_pokemons.base_hp + self.trainer_pokemons.IV_hp +
                                              self.trainer_pokemons.EV_hp/4)) + 10 + self.trainer_pokemons.level + self.trainer_pokemons.AV_hp)
        
        self.trainer_pokemons.final_attack = np.floor((self.trainer_pokemons.level/100.0) * (np.floor(2*self.trainer_pokemons.base_attack + self.trainer_pokemons.IV_attack+
                                              self.trainer_pokemons.EV_attack/4)) + 5 + self.trainer_pokemons.AV_attack)
        
        self.trainer_pokemons.final_defense = np.floor((self.trainer_pokemons.level/100.0) * (np.floor(2*self.trainer_pokemons.base_defense + self.trainer_pokemons.IV_defense+
                                              self.trainer_pokemons.EV_defense/4)) + 5 + self.trainer_pokemons.AV_defense)
        
        self.trainer_pokemons.final_ability = np.floor((self.trainer_pokemons.level/100.0) * (np.floor(2*self.trainer_pokemons.base_ability + self.trainer_pokemons.IV_ability+
                                              self.trainer_pokemons.IV_ability/4)) + 5 + self.trainer_pokemons.AV_ability)
        
        self.trainer_pokemons.final_resistance = np.floor((self.trainer_pokemons.level/100.0) * (np.floor(2*self.trainer_pokemons.base_resistance + self.trainer_pokemons.IV_resistance+
                                              self.trainer_pokemons.IV_resistance/4)) + 5 + self.trainer_pokemons.AV_resistance)
        
        self.trainer_pokemons.final_speed = np.floor((self.trainer_pokemons.level/100.0) * (np.floor(2*self.trainer_pokemons.base_speed + self.trainer_pokemons.IV_speed+
                                              self.trainer_pokemons.IV_speed/4)) + 5 + self.trainer_pokemons.AV_speed)
         
        
        self.trainer_pokemons.final_total = self.trainer_pokemons["final_total"] = (self.trainer_pokemons.final_hp + self.trainer_pokemons.final_attack
                                                                                + self.trainer_pokemons.final_defense + self.trainer_pokemons.final_ability +
                                                                                self.trainer_pokemons.final_resistance + self.trainer_pokemons.final_speed)
        
        # utils sector
        self.trainer_pokemons.final_charm = np.floor(self.trainer_pokemons.base_charm * (1+self.trainer_pokemons.level/25))
        self.trainer_pokemons.final_search = np.floor(self.trainer_pokemons.base_search * (1+self.trainer_pokemons.level/25))
        self.trainer_pokemons.final_fly = np.floor(self.trainer_pokemons.base_fly * (1+self.trainer_pokemons.level/25))
        self.trainer_pokemons.final_dig = np.floor(self.trainer_pokemons.base_dig * (1+self.trainer_pokemons.level/25))
        self.trainer_pokemons.final_picking = np.floor(self.trainer_pokemons.base_picking * (1+self.trainer_pokemons.level/25))
        self.trainer_pokemons.final_swim = np.floor(self.trainer_pokemons.base_swim * (1+self.trainer_pokemons.level/25))
        self.trainer_pokemons.final_shaking = np.floor(self.trainer_pokemons.base_shaking * (1+self.trainer_pokemons.level/25))
        self.trainer_pokemons.total_final_utils = (self.trainer_pokemons.final_charm + self.trainer_pokemons.final_search + self.trainer_pokemons.final_fly +
                                                   self.trainer_pokemons.final_dig + self.trainer_pokemons.final_picking + self.trainer_pokemons.final_swim +
                                                   self.trainer_pokemons.final_shaking)
        
        self.save_state()
        print("calculated and saved!")
        
        
    def add_pokemon_to_trainer(self, pokemon_name, pokemon_level, pokemon_sex):
        pokemon_entry = self.pokedex[self.pokedex.pk_name==pokemon_name].iloc[0]
        pokemon_entry["pk_sex"] = pokemon_sex
        pokemon_entry["level"] = pokemon_level
        pokemon_entry["experience"] = pokemon_level**3
        self.trainer_pokemons = self.trainer_pokemons.append(pokemon_entry, ignore_index=True)
        self.refresh_trainer_pokemons()
        
        
    def generate_wild_pokemon(self, pk_name, level, pk_sex):
        pk_entry = self.pokedex[self.pokedex.pk_name == pk_name].iloc[0]
        pk_entry["pk_sex"] = pk_sex
        pk_entry["level"] = level
        pk_entry["experience"] = level**3
        pk_entry.IV_hp =random.randint(0,self.MAX_IV)
        pk_entry.IV_attack=random.randint(0,self.MAX_IV)
        pk_entry.IV_defense=random.randint(0,self.MAX_IV)
        pk_entry.IV_ability=random.randint(0,self.MAX_IV)
        pk_entry.IV_resistance=random.randint(0,self.MAX_IV)
        pk_entry.IV_speed=random.randint(0,self.MAX_IV)
        	
        pk_entry.total_IV = (pk_entry.IV_hp + pk_entry.IV_attack + pk_entry.IV_defense + pk_entry.IV_ability +
                    pk_entry.IV_resistance + pk_entry.IV_speed)
        
        pk_entry.final_hp = np.floor((pk_entry.level/100.0) * (int(2*pk_entry.base_hp + pk_entry.IV_hp)) + 10 + self.trainer_pokemons.level)
        
        pk_entry.final_attack = np.floor((pk_entry.level/100.0) * (int(2*pk_entry.base_attack + pk_entry.IV_attack)) + 5 )
        
        pk_entry.final_defense = np.floor((pk_entry.level/100.0) * (int(2*pk_entry.base_defense + pk_entry.IV_defense)) + 5 )
        
        pk_entry.final_ability = np.floor((pk_entry.level/100.0) * (int(2*pk_entry.base_ability + pk_entry.IV_ability)) + 5 )
        
        pk_entry.final_resistance = np.floor((pk_entry.level/100.0) * (int(2*pk_entry.base_resistance + pk_entry.IV_resistance)) + 5)
        
        pk_entry.final_speed = np.floor((pk_entry.level/100.0) * (int(2*pk_entry.base_speed + pk_entry.IV_speed)) + 5)
         
        
        pk_entry.final_total  = (pk_entry.final_hp + pk_entry.final_attack + pk_entry.final_defense + pk_entry.final_ability + 
                                 pk_entry.final_resistance + pk_entry.final_speed)
        pk_entry.pk_sex = pk_sex
        self.wild_pokemon = pk_entry
        
        
    def start_training_taks(self, trainer_pokemon_index):
        start_time = datetime.now().timestamp()
        pokemon_name = self.trainer_pokemons.pk_name.iloc[trainer_pokemon_index]
#         bonues_dis = [0,0,0,0,0,1,1,1,2,4]
#         random_EV_bonuses_array = [random.choice(bonues_dis) for i in range(6)]
        results = "training hard!"
        pokemon_entry=trainer_pokemon_index
        pokemon_level = self.trainer_pokemons.level.iloc[trainer_pokemon_index]
        pokemon_sex = self.trainer_pokemons.pk_sex.iloc[trainer_pokemon_index]
        self.new_task_map = pd.DataFrame(dict(trainer_name = self.trainer_name, pokemon_entry=trainer_pokemon_index, pokemon_name=pokemon_name,
                                              pokemon_level = pokemon_level, task_type="training", pokemon_sex = pokemon_sex,
                                              starting_time = start_time, end_time = start_time + self.TASK_DUR, completed = 0,
                                              results=results), index=[len(self.tasks_csv)])
        self.tasks_csv = self.tasks_csv.append(self.new_task_map, sort=False)

        
        
    def start_task(self, trainer_pokemon_index, task_type):
        if (trainer_pokemon_index in list(self.tasks_csv.pokemon_entry)):
            print("{} is already doing a task".format(self.trainer_pokemons.pk_name.iloc[trainer_pokemon_index]))
            return
        
        if task_type == "t":
            task_type = "training"
        if task_type == "p":
            task_type = "wilderness_pokemon"
        if task_type == "s":
            task_type = "search"
            
        if task_type == "training":
            self.start_training_taks(trainer_pokemon_index)
        else:
            start_time = datetime.now().timestamp()
            #TODO, change the next line
            message = ""
            reward_item_0 = np.nan
            bonus_to_roll = np.nan
            cr = np.nan
            enemy_level = np.nan
            enemy_sex = np.nan
            enemy_pokemon = np.nan
            item_0_quantity = np.nan
            if task_type == "wilderness_pokemon":
                message += "VS "
                sample = self.wilderness_pokemons.sample().iloc[0]
                enemy_level = random.randint(int(sample.level.split("-")[0]), int(sample.level.split("-")[-1]))
                message += sample.pk_name + ": " + str(enemy_level)
                enemy_sex = "M" if random.random() < sample.male_chance else "F"
                enemy_pokemon = sample.pk_name

            if task_type == "search":
                message += "searching for "
                sample = self.wilderness_items.sample().iloc[0]
                item_0_quantity = str(random.randint(int(sample.quantity.split("-")[0]), int(sample.quantity.split("-")[-1])))
                message += sample.item_name + ": " + item_0_quantity
                reward_item_0 = sample.item_name
                bonus_to_roll = int(self.trainer_pokemons.final_search.iloc[trainer_pokemon_index])
                cr = sample.cr
            if task_type == "rest":
                message += "resting"

            pokemon_name = self.trainer_pokemons.pk_name.iloc[trainer_pokemon_index]
            pokemon_level = self.trainer_pokemons.level.iloc[trainer_pokemon_index]
            pokemon_sex = self.trainer_pokemons.pk_sex.iloc[trainer_pokemon_index]
            self.new_task_map = pd.DataFrame(dict(trainer_name = self.trainer_name, pokemon_entry=trainer_pokemon_index,pokemon_name=pokemon_name,
                                                  pokemon_level = pokemon_level, task_type=task_type, pokemon_sex = pokemon_sex,
                                    starting_time = start_time, end_time = start_time + self.TASK_DUR, completed = 0, location=self.location_name,
                                                  message=message, bonus_to_roll=bonus_to_roll, cr=cr, enemy_sex =enemy_sex, enemy_level=enemy_level,
                                                  item_0_quantity=item_0_quantity, enemy_pokemon=enemy_pokemon, reward_item_0=reward_item_0), index=[len(self.tasks_csv)])
            self.tasks_csv = self.tasks_csv.append(self.new_task_map, sort=False)
        self.save_state()
        
        
    def add_exp_to_pokemon(self, trainer_pk_index, exp_amount):
        pk_name = self.trainer_pokemons.iloc[trainer_pk_index].pk_name
        previous_level = int(self.trainer_pokemons.iloc[trainer_pk_index].level)
        print("{} gained {} exp.".format(pk_name, np.floor(exp_amount)))
        self.trainer_pokemons.at[trainer_pk_index, "experience"] += exp_amount
        self.trainer_pokemons.at[trainer_pk_index, "level"] = int((self.trainer_pokemons.iloc[trainer_pk_index].experience)**(1/3.0))
        if int(self.trainer_pokemons.iloc[trainer_pk_index].level) > previous_level:
            print("{} has levels up and is now level {}".format(pk_name, int(self.trainer_pokemons.iloc[trainer_pk_index].level)))
            self.refresh_trainer_pokemons()
        else:
            self.save_state()
        
    def collect_training_results(self, pk_entry):
        training_array = [0,0,0,0,0,1,1,1,2,4]
        name = self.trainer_pokemons.pk_name.iloc[pk_entry]
        print("{} training is complete".format(name))
        
        old_val = self.trainer_pokemons.EV_hp.iloc[pk_entry]
        new_val = min(old_val + random.choice(training_array), self.MAX_EV)
        if new_val > old_val:
            self.trainer_pokemons.at[pk_entry, "EV_hp"] = new_val
            print("{} EV_hp has been raised by {} points!".format(name, new_val - old_val))
            
        old_val = self.trainer_pokemons.EV_attack.iloc[pk_entry]
        new_val = min(old_val + random.choice(training_array), self.MAX_EV)
        if new_val > old_val:
            self.trainer_pokemons.at[pk_entry, "EV_attack"] = new_val
            print("{} EV_attack has been raised by {} points!".format(name, new_val - old_val))
            
        old_val = self.trainer_pokemons.EV_defense.iloc[pk_entry]
        new_val = min(old_val + random.choice(training_array), self.MAX_EV)
        if new_val > old_val:
            self.trainer_pokemons.at[pk_entry, "EV_defense"] = new_val
            print("{} EV_defense has been raised by {} points!".format(name, new_val - old_val))
            
        old_val = self.trainer_pokemons.EV_ability.iloc[pk_entry]
        new_val = min(old_val + random.choice(training_array), self.MAX_EV)
        if new_val > old_val:
            self.trainer_pokemons.at[pk_entry, "EV_ability"] = new_val
            print("{} EV_ability has been raised by {} points!".format(name, new_val - old_val))
            
        old_val = self.trainer_pokemons.EV_resistance.iloc[pk_entry]
        new_val = min(old_val + random.choice(training_array), self.MAX_EV)
        if new_val > old_val:
            self.trainer_pokemons.at[pk_entry, "EV_resistance"] = new_val
            print("{} EV_resistance has been raised by {} points!".format(name, new_val - old_val))
            
        old_val = self.trainer_pokemons.EV_speed.iloc[pk_entry]
        new_val = min(old_val + random.choice(training_array), self.MAX_EV)
        if new_val > old_val:
            self.trainer_pokemons.at[pk_entry, "EV_speed"] = new_val
            print("{} EV_speed has been raised by {} points!".format(name, new_val - old_val))
            
#         self.MAX_EV
        self.refresh_trainer_pokemons()
        
        
    def show_ongoing_taks(self):
        self.tasks_csv = self.tasks_csv[self.tasks_csv.completed == 0]
        for i, r in self.tasks_csv.iterrows():
            now = datetime.now().timestamp()
            total_time_modifer = 100/(r.end_time - r.starting_time)
            time_passed = min(100, int((now - r.starting_time)*total_time_modifer))
            time_left = int(100-time_passed)
            print(str(r.pokemon_entry) + " " + str(r.pokemon_name) +" " + str(r.pokemon_sex)+ " " + str(int(r.pokemon_level)) + " "
                  + time_passed*"#" + "-"*time_left + " " + str(r.message) + " " + str(r.enemy_sex))
        
    def collect_finished_tasks(self):
        self.tasks_csv = self.tasks_csv[self.tasks_csv.completed == 0]
        self.finish_tasks = self.tasks_csv[self.tasks_csv.end_time < datetime.now().timestamp()]
        self.tasks_csv = self.tasks_csv[self.tasks_csv.end_time >= datetime.now().timestamp()]
        self.tasks_csv.to_csv(self.trainer_path + "/tasks.csv", index=[0])
        
        for i, r in self.finish_tasks[self.finish_tasks.task_type == "search"].iterrows():
            random_roll = random.randint(1, 20)
            print(r.pokemon_name)
            print("roll: {}".format(random_roll))
            did_pass = 1 if ((random_roll + r.bonus_to_roll) >= int(r.cr) ) else 0
            print("passed!") if did_pass else print("failed")
            gained_items = [r.reward_item_0]
            for g in gained_items:
                item_name = g
                print(item_name)
                print("-"*90)
                item_quantity = int(r.item_0_quantity)
                if not (len(self.trainer_items.index[self.trainer_items.item_name == item_name])):
                    self.trainer_items = self.trainer_items.append(self.item_dex[self.item_dex.item_name==item_name], sort=False, ignore_index = True)
                    item_id = self.trainer_items.index[self.trainer_items.item_name == item_name][0]
                    self.trainer_items.at[item_id, "quantity"] = 0
                    previous_quantity = 0
                else:
                    item_id = self.trainer_items.index[self.trainer_items.item_name == item_name][0]
                    previous_quantity = currenct_game.trainer_items.quantity.iloc[item_id]
                    
                self.trainer_items.at[item_id, "quantity"] = previous_quantity + item_quantity
                
            self.save_state()
            
        for i, r in self.finish_tasks[self.finish_tasks.task_type == "wilderness_pokemon"].iterrows():
            self.generate_wild_pokemon(pk_name = r.enemy_pokemon, level = r.enemy_level, pk_sex=r.enemy_sex)
            # created in memory self.wild_pokemon
            trainer_pokemon = self.trainer_pokemons.iloc[r.pokemon_entry]
            own_roll = random.randint(1, 50)
            enemy_roll = random.randint(1, 50)
            print(r.pokemon_name)
            print("{} strength: {}".format(trainer_pokemon.pk_name, trainer_pokemon.final_total))
            print("own roll: {}".format(own_roll))
            print("VS")
            print("{} strengh {}".format(self.wild_pokemon.pk_name, np.floor(self.wild_pokemon.final_total)[0]))
            print("enemy roll: {}".format(enemy_roll))
            did_pass = 1 if ((own_roll + trainer_pokemon.final_total) >= enemy_roll + np.floor(self.wild_pokemon.final_total).iloc[0]) else 0
            print("{} deafeated {}".format(trainer_pokemon.pk_name, self.wild_pokemon.pk_name)) if did_pass else print("{} fainted".format(r.pokemon_name))
            if did_pass:
                experience_yield = np.floor(self.wild_pokemon.base_total**3/650**2) * self.wild_pokemon.level
                level_dif_modifier = np.floor((2*trainer_pokemon.level+10)**2.5/(trainer_pokemon.level + self.wild_pokemon.level + 10)**2.5)
                delta_exp = experience_yield * level_dif_modifier * trainer_pokemon.LR * 0.2
                self.add_exp_to_pokemon(r.pokemon_entry, delta_exp)
                print("checking pokedex...")
                time.sleep(2.5)
                if len(currenct_game.trainer_pokemons[(currenct_game.trainer_pokemons.pk_name == self.wild_pokemon.pk_name) &
                                                      (currenct_game.trainer_pokemons.pk_sex == self.wild_pokemon.pk_sex)]) == 0:
                    print("pokedex: \"New Pokemon!\"")
                    pokeballs = self.trainer_items[currenct_game.trainer_items.item_name=="pokeball"]
                    left_pokeballs = pokeballs.quantity[0]
                    item_id = pokeballs.index[0]
                    catch_rate = pokeballs.catch_rate[0] + trainer_pokemon.final_charm
                    if (left_pokeballs > 0):
                        print("you got {} pokeballs left, trying to catch {}".format(left_pokeballs, r.enemy_pokemon))
                        time.sleep(2.5)
                        self.trainer_items.at[item_id, "quantity"] = left_pokeballs - 1
                        pokeball_roll = random.randint(1, 20)
                        opponent_roll = random.randint(1, 20)
                        if (pokeball_roll + catch_rate > opponent_roll + r.enemy_level):
                            print("You caught {}!!!".format(r.enemy_pokemon))
                            self.trainer_pokemons = self.trainer_pokemons.append(self.wild_pokemon, ignore_index=True)
                            self.refresh_trainer_pokemons()
                        else:
                            print("{} escaped".format(r.enemy_pokemon))
                    else:
                        print("You are out of pokeballs!")
                else:
                    print("You got {} already".format(r.enemy_pokemon))
            else:
                currenct_game.start_task(r.pokemon_entry, "rest")
                print("{} is resting at the pokecenter".format(r.pokemon_name))
            self.save_state()
            
        for i, r in self.finish_tasks[self.finish_tasks.task_type == "training"].iterrows():
            self.collect_training_results(r.pokemon_entry)

#             for g in gained_items:
#                 item_name = g.split(":")[0].strip()
#                 print(item_name)
#                 print("-"*90)
#                 item_quantity = int(g.split(":")[-1].strip())
#                 item_id = self.trainer_items.index[self.trainer_items.item_name == item_name][0]
#                 previous_quantity = currenct_game.trainer_items.quantity.iloc[item_id]
#                 self.trainer_items.at[item_id, "quantity"] = previous_quantity+item_quantity
                
#             self.trainer_items.to_csv(self.trainer_path + "/items.csv", index=[0])
            
    def rush_all_tasks(self):
        #df[col].values[:] = 0
        self.tasks_csv["end_time"].values[:] = datetime.now().timestamp()
                


In [310]:
currenct_game = game(currenct_trainer_path, location_path)
# currenct_game.add_pokemon_to_trainer("chimchar", 5, "M")
#viridian forest

In [376]:
# currenct_game.refresh_trainer_pokemons()
for i in range(4):
    currenct_game.start_task(i, "p")
    
currenct_game.collect_finished_tasks()
currenct_game.show_ongoing_taks()

chimchar is already doing a task
egypt_riolu is already doing a task
neferkitty is already doing a task
egypt_numel is already doing a task
chimchar
chimchar strength: 106.0
own roll: 13
VS
rattata strengh 53.0
enemy roll: 32
chimchar deafeated rattata
chimchar gained 30.0 exp.
checking pokedex...
pokedex: "New Pokemon!"


AttributeError: 'Series' object has no attribute 'charm'

In [370]:
# currenct_game.rush_all_tasks()
currenct_game.collect_finished_tasks()
currenct_game.show_ongoing_taks()

0 chimchar M 9 #########################################################################################----------- nan nan
1 egypt_riolu M 3 #########################################################################################----------- nan nan
2 neferkitty M 5 #########################################################################################----------- nan nan
3 egypt_numel F 3 #########################################################################################----------- nan nan


In [373]:
currenct_game.trainer_pokemons.final_total.head()

0    106.0
1     55.0
2     68.0
3     59.0
Name: final_total, dtype: float64